## Notebook Setup

In [26]:
import math
import toolz as tz
import numpy as np

def _rand_list(size:int=20, max_val: int = 100) -> list[int]:
    return np.random.randint(10, max_val,size=size) #.tolist()

def _swap(A: list[int], i:int, j:int) -> None:
    _ = A[i]
    A[i] = A[j]
    A[j] = _

## Algorithmic Analysis

### Asymptotic bounds

**$\Theta$-notation** gives a bounded fit to a function. $f(n) = \Theta(g(n))$ if there exist positive constants $n_0$, $c_1$, and $c_2$ such that, for $ n > n_0, c_1g(n) \leq f(n)  \leq c_2g(n)$<br>
**O-notation** gives an upper bound for a function.
$f(n) = O(g(n)) if there are positive constants $n_0$ and $c$ such that, for $n > n_0, f(n) < cg(n)$ <br>
**$\Omega$-notation** gives a lower bound for a function. $f(n) = \Omega(g(n))$ if there are positive constants $n_0$ and $c$ such that, for $n > n_0, f(n) > cg(n)$

#### Growth of Common Functions
$log(n) < n < nlog(n) < log^k(n) < n^k < k^n < n! $

# Comparison-based Sorting

## Insertion Sort

Walk over the elements in the input array and iteratively put them into another array in the correct order.

#### In-place sorting
Keep the left side of the array sorted and move to the right.  This **keeps memory constant** and allows for a **binary search** to locate the insertion index but it will require lots of **'shift right by 1' operations**.
#### Insert into a linked list
Allocate a new linked list to store the sorted values.  This requires more memory and doesn't allow for binary searches during insertion but it does prevent the need to shift items by 1 during insertion.

In [ ]:
def _shift_right(array: list, s: int, e: int):
    # going backwards allows us to use the space of the item being inserted as swap
    for i in range(e,s,-1):
        array[i] = array[i-1]
    return array

def insertion_sort(input: list, in_place:bool=True):
    _in = input if in_place else input.copy()
    for j in range(1,len(_in)):
        # find correct location for _in[j] in the array _in[0:j]
        for i in range(0,j):
            if _in[j] < _in[i]:
                v = _in[j]
                _shift_right(_in, i, j)
                _in[i] = v
                break
    return _in

In [ ]:
A = _rand_list()

display(A)
insertion_sort(A)

## Merge Sort

Split the input array in half, sort those halves, and then merge the sorted halves together.

The tricky part will be the merge logic.  Since we know that the two arrays being merged are already sorted, we can incrementally move across both arrays and only compare a item to a (likely) small subset of the other values in the other array.

I have used this merging logic elsewhere to simplify processes (I think when merging potentially overlapping time series data for Prima).

In [ ]:
def merge(a: list[int], b: list[int]) -> list[int]:
    rval, a_idx, b_idx = [],0,0

    while a_idx < len(a) and b_idx < len(b):
        if a[a_idx] < b[b_idx]:
            rval.append(a[a_idx])
            a_idx += 1
        else:
            rval.append(b[b_idx])
            b_idx += 1
    # extend works with np.array or list[int], whereas += is elementwise for np
    rval.extend(a[a_idx:] if a_idx < len(a) else b[b_idx:])
    return rval


def merge_sort(a: list[int]) -> list[int]:
    if len(a) == 1:
        return a
    else:
        i = int(len(a) / 2)
        return merge(merge_sort(a[:i]), merge_sort(a[i:]))

In [ ]:
A = _rand_list()

display(A)
merge_sort(A)

## Heapsort

Heapsort works by:
- Creating a max-heap from an unsorted array
- Iteratively pulling off the root, inserting it into our sorted array, and re-establishing the heap property in the array
- The can be done in-place by:
  - Storing a `heap-size` variable
  - Swaping the value at the root (the highest value) with the last element of the heap.
  - decrementing `heap-size` (so that the previous root isn't in the heap)
  - Moving the new value at the root to the correct location (push it down the tree)

### Heap
Heaps are nearly complete binary trees that have the property that either a node is always $\geq$ its parent (min-heap) or $\leq$ its parent (max-heap).

In [ ]:
def _height_at(i: int) -> int:
    "Returns the height of the element at index i, where the root is at level 0"
    return math.floor(math.log2(i+1))


def _left(i):
    return (2*i) + 1

def _right(i):
    return 2*(i+1)

def _parent(i):
    return int((i-1)/2)


def _max_heapify(heap: list[int], i: int, size: int) -> list[int]:
    "Percolate the element at index down the descendants branch so that the max heap property is maintained"
    end = size - 1
    l, r, mx = _left(i), _right(i), i
    if l <= end and heap[l] > heap[i]:
        mx = l
    if r <= end and heap[r] > heap[mx]:
        mx = r

    if mx == i:
        return heap
    else:
        _swap(heap, mx,i)
        return _max_heapify(heap, mx, size)


# Exercise : Determing indices of all leaf nodes for heap
# 0
# 1,2
# 3,4 5,6
# 7,8 9,10 11,12 13,14

# heap size  | leaf indices
# 1 | 0
# 2 | 1
# 3 | 1,2
# 4 | 2,3
# 5 | 2,3,4
# 6 | 3,4,5
# 7 | 3,4,5,6
# 8 | 4,5,6,7
# 9 | 4,5,6,7,8

# From this pattern, it looks like the all indices >= floor(size/2) are leaf nodes

# So to build a max heap, we take a recursive approach by building up heaps from the bottom,
# essentially building heaps by composing smaller heaps.  Leaf nodes are already heaps, so 
# we start from the first non-heap index and build a heap from that, then iterate to the root

def build_max_heap(a: list[int]) -> list[int]:
    for i in range(int(len(a)/2), -1, -1):
        _max_heapify(a, i, len(a))
    return a


def print_binary_tree(tree: list):
    pad_iter = tz.iterate(lambda x: x*2 + 2, 1) # assume 1 space between leaves, 2 character elements
    _pad = [next(pad_iter) for _ in range(_height_at(len(tree)-1)+2)][-1]  # +1 for 0 indexed height calc, +1 since range() excludes end point 
    s, prev_height = "", -1
    for i in range(len(tree)):
        _height = _height_at(i)
        if _height > prev_height:
            s +=  "\n" if _height else ""
            prev_height = _height
            _pad = int((_pad / 2)) - 1
            s += (" "*int(_pad/2))
        else:
            s += (" "*_pad)
        s += str(tree[i])
    print(s)


In [ ]:
A = _rand_list()
_heap = build_max_heap(A)
print_binary_tree(_heap)

### In-place sorting with a heap

In [ ]:
def heap_sort(a: list[int]) -> list[int]:
    size = len(a)
    heap = build_max_heap(a)

    while size > 0:
        _swap(heap, size-1,0)
        size -= 1
        _max_heapify(heap, 0, size)

    return heap


In [ ]:
A = _rand_list()
display(A)
heap_sort(A)

## Quicksort

The divide-and-conquer process for sorting a typical subarray A:
- **Divide:**
  - Partition and rearrange A into two (possibly empty) subarrays $A[0:q]$ and $A[q+1:]$ such that each element of $A[0:q]$ is less than or equal to $A[q]$, which is, in turn, less than or equal to each element of $A[q+1:]$ Compute the index $q$ as part of this partitioning procedure.
- **Conquer:**
  - Sort the two subarrays $A[0:q]$ and $A[q+1:]$ by recursive calls to quicksort.

### Partition
- Identifying 3 indices
  - $r$, such that $A[r]$ is our reference element
  - $i$, the index that separates our $\leq$ and $\geq$ partitions.
  - $j$, the index that marks the end of our $\geq$ partition
  - (Typically, these are $r=len(A)-1, i=0,j=0$)
- Iteratively, increment $j$ and move elements and/or update $i$ to maintain our $\leq$ and $\geq$ partitions

![QuickSort Partition](resources/images/quicksort_partition.png)

### Randomized Quicksort

The worst-case behavior for quicksort occurs when the array is already sorted, since the partitioning routine produces one subproblem with $n-1$ elements and one with 0 elements.  This is a common scenario and results in $O(n^2)$ runtime, so ideally we could modify the algorithm to handle this particular scenario more efficiently.

We can achieve this by initially permuting the input so that it is randomized, specifically changing a sorted array so that it is no longer sorted.  While we could permute the entire array, a more efficient randomization is to choose a random element at the initial reference point (instead of the last element).  In practice, this is simple as we just swap the value at the randomly selected index with the last element of the array and then call our standard quicksort routine.

In [ ]:
# A[i] is first element of gte partition
def qs_partition(A: list[int], i: int, ri: int) -> list[int]:
    # i=midpoint index, j = partition end index, ri = reference index 
    j = i
    while j < ri:
        if A[j] < A[ri]:
            _swap(A,i,j)
            i += 1
        j +=1
    _swap(A,i,ri) # move A[ri] to correct position (midpoint)
    return i

def _quicksort(A: list[int], i: int, j: int) -> list[int]:
    if i < j:
        q = qs_partition(A, i, j)
        _quicksort(A, i, q-1)
        _quicksort(A, q+1, j)


def quick_sort(A: list[int], randomize:bool=True) -> list[int]:
    if randomize:
        i = np.random.randint(0,len(A)-1)
        _swap(A, i, len(A)-1)
        
    _quicksort(A, 0, len(A) -1)

In [ ]:
A = _rand_list()
display(A)
quick_sort(A)  # inplace sort
display(A)

# Linear Time Sorting

In a comparison sorts, we use only comparisons between elements to gain order
information about an input sequence.  This ultimately limits the expected run-time of our algorithms to $O(nlogn)$.  However, we can lower this bound by making some additional assumptions and then exploiting that knowledge to avoid making direct comparisons between elements.

## Counting Sort

Assume that the values in our array are all in the range $[0,k]$.  Then we can perform a linear time ($O(n+k)$) sort, assuming $k = O(n)$, by counting the number of elements in the array that are $leq$ each value in $[0,k]$ (create a new array of size $k$ to hold this information).  This information allows for direct placement of elements into their sorted order.

For unsorted array $A$ and maximum element value of $k$
- Create a new counter array, $C$, of size $k$ and a new output array, $RESULT$, of size $len(A)$
- Populate $C$ so that $C[i]$ = number of items with a value of $i$
  - For each $a$ in $A$, increment $C[a]$
- Update $C$ so that $C[i]$ contains the number of items with a value $\leq i$
  - For i in $[1, k]$, $C[i] = C[i] + C[i-1]$  (updates in place)
- Use $C$ to place each element of $A$ into the correct position in a new array (same length as $A$)
  - For each $a$ in $A$, place $a$ into $RESULT[C[a]]$ and decrement $C[a]$
    - We decrement to handle duplicate elements in our array

In [ ]:
def counting_sort(A: list[int], max_value: int) -> list[int]:
    B, C = np.zeros(len(A), dtype=np.int32), np.zeros(max_value, dtype=np.int32)
    # Count number of instances of a particular value
    for a in A:
        C[a] += 1
    # Calculate the number of elements with a value <= the index value
    for i in range(1, max_value):
        C[i] = C[i] + C[i-1]
    # Assign the elements to the correct location in the output array
    for a in A:
        B[C[a]-1] = a  # 0 indexed
        C[a] -= 1
    return B

In [ ]:
A = _rand_list()
display(A)
display(counting_sort(A, 100))

## BucketSort

Bucket sort assumes that the input is drawn from a uniform distribution and has an average-case running time of $O(n)$. Like counting sort, bucket sort is fast because it assumes something about the input. Whereas counting sort assumes that the inputconsists of integers in a small range, bucket sort assumes that the input is generated by a random process that distributes elements uniformly and independently over the interval $[0, 1)$.

Bucket sort divides the interval $[0, 1)$ into n equal-sized subintervals, or buckets,and then distributes the n input numbers into the buckets. Since the inputs are uniformly and independently distributed over $[0, 1)$ we do not expect many numbers to fall into each bucket. To produce the output, we simply sort the numbers in each bucket and then go through the buckets in order, listing the elements in each.

In [ ]:
def bucket_sort(A: list[float]) -> list[float]:
    buckets = [[] for _ in range(10)]
    # Partition elements into buckets
    for a in A:
        # .099999 -> 0, 0.199999 -> 1, ..., 0.999 -> 9
        i = int(a*10)
        buckets[i].append(a)
    # Sort each bucket
    for b in buckets:
        insertion_sort(b)
    # Concat the buckets sequentially
    return list(tz.concat([b for b in buckets]))

In [ ]:
A = np.random.rand(20)
display(A)
display(bucket_sort(A))

# Order statistics

We can determine the nth smallest number by employing the partitioning strategy from quick sort to iteratively divide the problem.

In [ ]:
def nth_smallest(A: list[int], nth: int) -> int:
    if len(A) <= 1:
        return A[0]
    # randomized qs_partition
    i = np.random.randint(len(A)-1)
    _swap(A, i, len(A)-1)
    q = qs_partition(A, 0, len(A)-1)
    if nth == q+1:
        return A[q]
    elif nth > q:
        return nth_smallest(A[q+1:], nth-(q+1))
    return nth_smallest(A[0:q], nth)


for i in range(1,10):
    A = _rand_list(20)
    a = nth_smallest(A, i)
    b = insertion_sort(A, in_place=False)
    assert a == b[i-1]

# Graphs

A graph is a data structure consisting of *vertices* (also known as nodes) and *edges* (links between vertices). Usually this is denoted as G = <V, E>, where V is a set of vertices, and E is a set of edges.

#### Edge properties
An *Edge* represents a relationship between nodes/vertices and has the following properties:
  - A pair of nodes/vertices
  - A relation weight term (optional)
  - A direction (optional)

#### Vertex properties
A vertex/node represents entities (e.g. people in a social network) and has the equivalent of a pointer to an entity object.  As a node in the graph, it has the properties:
- out-degree - the number of edges leaving the vertex
- in-degree - the number of edges entering the vertex
- degree - out-degree + in-degree


#### Graph descriptors:
- **Simple** if there is no more than one edge between any two vertices, **multigraph** otherwise
- **Cyclic** if there is a cycle in the graph, **acyclic** otherwise
- **Directed** if the edges have directions (e.g. is_parent)
- **Weighted** if the edges have weights
- **Connected** if there is a path between any two vertices
- **Dense** if number of edges is close to $len(V)^2$, sparse otherwise
- **Complete** if every vertex is directly connected (an edge exists that contains the vertex pair) to every other vertex.

#### Miscellaneous Terms
A **path** is a sequence of distinctive vertices connected by edges, and its length is the number of edges within it (weighted sum if edges have weights).  A graph is connected, if there is a path between any two vertices. A tree is a connected graph without cycles

A **subgraph** of a graph G is a graph whose vertices and edges are subsets of the vertices and edges of G.

A **spanning tree** is a subgraph which is a tree and contains all vertices of the graph.


# Tree Data Structures

 A tree is a directed acyclic graph in which one node has no parent (the **root**) and every other node has *exactly one parent*.

#### Tree Descriptors
- **Binary** if each node has atmost 2 children
- **Full binary** if each node has exactly zero or two children
- **Complete** binary tree if all internal nodes have 2 children, with the possible exception of the bottom level, which is filled from left to right


#### Miscellaneous terms
- **leaves** are nodes with no children
- **internal nodes** are nodes that are not the root and are not leaves
- Nodes with the same parent are called **siblings**
- The **depth** of a node is the number of edges from the root to the node.
- The **height** of a node is the number of edges from the node to the deepest leaf
- **Tree height** is the height of the root.


## Binary Search Trees

A binary search tree is always stored in such a way as to satisfy the
binary-search-tree properties:

- If Y is a node in the left subtree of X, then $Y_{key} \leq X_{key}$
- If Y is a node in the right subtree of X, then $Y_{key} \geq X_{key}$


### Traversal Types
- **inorder tree walk** - processes the values in its left subtree, then the root, then values in its right subtree.
- **preorder** - processes the root before the values in either subtree
- **postorder** - processes the root after the values in its subtrees.

In [2]:
# Build Tree
def _node(key: int, parent: dict, left:dict = None, right:dict=None) -> dict:
    return {"parent": parent, "key": key, "left": left, "right": right}

# insert into tree

def tree_insert(root: dict, new_key: int) -> None:
    if root is None:
        return _node(new_key, None)
    y, z, dr = root, None, None
    # traverse down to the bottom of the tree
    while y:
        dr = "right" if new_key >= y['key'] else "left"
        z, y = y, y[dr]
    new_node = _node(new_key, z)
    z[dr] = new_node
    return root


def build_bsearch_tree(keys: list[int]) -> dict:
    root, _ = None, np.random.shuffle(keys)
    for k in keys:
        root = tree_insert(root, k)
    return root

A = _rand_list(10)
display(A)
root = build_bsearch_tree(A)
root

array([29, 61, 41, 10, 52, 83, 65, 61, 86, 21])

{'parent': None,
 'key': 65,
 'left': {'parent': {...},
  'key': 10,
  'left': None,
  'right': {'parent': {...},
   'key': 61,
   'left': {'parent': {...},
    'key': 29,
    'left': {'parent': {...}, 'key': 21, 'left': None, 'right': None},
    'right': {'parent': {...},
     'key': 52,
     'left': {'parent': {...}, 'key': 41, 'left': None, 'right': None},
     'right': None}},
   'right': {'parent': {...}, 'key': 61, 'left': None, 'right': None}}},
 'right': {'parent': {...},
  'key': 83,
  'left': None,
  'right': {'parent': {...}, 'key': 86, 'left': None, 'right': None}}}

In [3]:
# in order traversal of binary seach tree yields a sorted list
def traverse(node: dict, vals=[]):
    if l := node['left']:
        traverse(l, vals)
    # print(node['key'], va)
    vals.append(node["key"])
    if r:= node['right']:
        traverse(r, vals)
    return vals

traverse(root)

[10, 21, 29, 41, 52, 61, 61, 65, 83, 86]

In [7]:
def tree_search(node: dict, k: int) -> dict:
    if node:
        if node['key'] == k:
            return node
        elif node['key'] > k:
            return tree_search(node['left'], k)
        return tree_search(node['right'], k)


def node_ancestry(node: dict, k: int, path=[]):
    if node:
        if node['key'] == k:
            return path + [k]
        elif node['key'] > k:
            return node_ancestry(node['left'], k, path + [node['key']])
        return node_ancestry(node['right'], k, path + [node['key']])
    return None


def path_to(node: dict, k: int, path=""):
    if node:
        if node['key'] == k:
            return path
        elif node['key'] > k:
            return path_to(node['left'], k, path + "L")
        return path_to(node['right'], k, path + "R")
    return None


# To keep it balanced, probably want to move up the subtree with the most nodes instead of always moving up the one on the right
def remove_node(root: dict, k: int):
    if to_remove := tree_search(root, k):
        side = None if to_remove["parent"] is None else "left" if to_remove["parent"]["key"] > k else "right"
        if to_remove['left'] is None:
            if to_remove['right'] is None:
                if side:
                    to_remove["parent"][side] = None
            else:
                if side:
                    to_remove["parent"][side] = to_remove["right"]
                to_remove["right"]["parent"] = to_remove["parent"]
        elif to_remove['right'] is None:
            if side:
                to_remove["parent"][side] = to_remove["left"]
            to_remove["left"]["parent"] = to_remove["parent"]
        else:  # Has 2 children ... the dynasty is doomed
            # move X.right up and place X.left (subtree) at the bottom left of the X.right.left subtree
            if side:
                to_remove["parent"][side] = to_remove["right"]
            to_remove["right"]["parent"] = to_remove["parent"]
            min_node = to_remove["right"]  # find min node of left of new parent
            while min_node["left"]:
                min_node = min_node["left"]
            min_node["left"] = to_remove["left"]
            to_remove["left"]["parent"] = min_node["left"]
        
        if to_remove["parent"] is None:  # removed the root
            return to_remove["right"]
    return root


node_ancestry(root, 60)
# path_to(root, 85)


## Red-Black Trees

A red-black tree is a binary search tree with one extra bit of storage per node: its color, which can be either RED or BLACK. By constraining the node colors on any simple path from the root to a leaf, red-black trees ensure that no such path is more than twice as long as any other, so that the tree is approximately balanced.

A red-black tree is a binary tree that satisfies the following red-black properties:
1. Every node is either red or black.
2. The root is black.
3. Every leaf (NIL) is black.
4. If a node is red, then both its children are black.
5. For each node, all simple paths from the node to descendant leaves contain the same number of black nodes.


## AVL Trees

## B Trees

B-trees are balanced search trees designed to work well on disks or other direct-access secondary storage devices. B-trees are similar to red-black trees but they are better at minimizing disk I/O operations. Many database systems use B-trees, or variants of B-trees, to store information. B-trees differ from red-black trees in that B-tree nodes may have many children, from a few to thousands. That is, the “branching factor” of a B-tree can be quite large, although it usually depends on characteristics of the disk unit used. B-trees are similar to red-black trees in that every n-node B-tree has height $O(\log n)$. The exact height of a B-tree can be considerably less than that of a red-black tree,however, because its branching factor, and hence the base of the logarithm that expresses its height, can be much larger. Therefore, we can also use B-trees to implement many dynamic-set operations in time $O(\log n)$.

If an internal B-tree node $x$ contains $x.n$ keys, then $x$ has $x.n + 1$
children. The keys in node $x$ serve as dividing points separating the range of keys handled by x into $x.n + 1$ subranges, each handled by one child of $x$. When searching for a key in a B-tree, we make $(x.n + 1) comparisons to determine which subtree to search. The structure of leaf nodes differs from that of internal nodes.

![BTree](resources/images/Btree.png)

A B-tree has the following properties:
1. Every node x has the following attributes
   1. $x.n$, the number of keys currently stored in node $x$,
   2. the $x.n$ keys themselves, $x.key_1$, $x.key_2$, ... $x.key_n$, are stored in nondecreasing order
   3. $x.leaf$ , a boolean value recording if the node is a leaf
2. Each internal node x also contains $x.n + 1$ children
3. The keys $x.key_i$ separate the ranges of keys stored in each subtree, such that $all(x.c[0].keys) < x.keys[0] < all(x.c[1].keys) < x.keys[1] ...$
4. All leaves have the same depth, which is the tree’s height h.
5. Nodes have lower and upper bounds on the number of keys they can contain. We express these bounds in terms of a fixed integer $t \geq 2$ called the minimum degree of the B-tree:
   1. Every node other than the root must have at least $t-1$ keys. Every internal node other than the root thus has at least $t$ children. If the tree is nonempty, the root must have at least one key.
   2. Every node may contain at most $2t-1$ keys. Therefore, an internal node  may have at most $2t$ children. We say that a node is full if it contains  exactly $2t-1$ keys.


# Graphs

## Representations
Graphs are typically represented as adjacency matrices or adjacency lists, depending on the connection density.

An adjacency matrix, $X$, is an $nxn$ matrix where the cell $X_{i,j}$ contains the edge weight (0 if no edge exists) between node $i$ and $j$. Matrices allow for $O(n)$ random access to connection information but are very inefficient in memory (or cannot fit at all, if n = billion) if most weights are 0 (i.e. most nodes are not connnected to each other).

Alternatively, an adjacenct list contains a list for each node, where the list for node A contains all nodes that are adjacent to A.  Such a structure uses memory more efficiently for sparsely connected graphs, but at the cost of slower lookup.

In [32]:
def create_adjacencies(elements: np.ndarray, sparsity: float=0.2, as_matrix:bool=False) -> tuple[list[int]]:
    n = len(elements)
    rand_X = np.random.random((n,n))  # random n x n matrix
    X = (rand_X < sparsity)   # apply sparsity constraint
    if as_matrix:
        return X * 1  # replace True/False with 1 or 0
    # build adjacency lists
    rows, cols = X.nonzero()
    adj_lists = [[] for _ in range(n)]
    for i,row in enumerate(rows):
        adj_lists[row].append(cols[i])
    return adj_lists
    

In [33]:
nodes = _rand_list(10)
adj_lists = create_adjacencies(nodes, 0.2)
# adj_lists references nodes by index

[[1, 3, 7, 8],
 [6],
 [5, 8],
 [0, 6, 7, 9],
 [1, 9],
 [6],
 [7],
 [0, 9],
 [1, 5, 8],
 [0, 1, 5]]

In [ ]:
# def breadth_search(graph: list[list[int]], k: int) -> 